In [2]:
import yfinance as yf
import pandas as pd
import psycopg as pg
from daily_insert.config import DB_CONFIG

with pg.connect(**DB_CONFIG) as conn:
    with conn.cursor() as cur:
        cur.execute('SELECT ticker FROM tickers')
        tickers = cur.fetchall()

assets = [ticker[0] for ticker in tickers]

df_list = []
for asset in assets:
    data = yf.download(asset, interval='5m')
    data = data.droplevel(1, axis=1)
    data['ticker'] = asset
    df_list.append(data)

df = pd.concat(df_list, axis=0)
df = df.reset_index()
df['Volume'] = df['Volume'].astype(int)

from io import StringIO

# Create buffer
buffer = StringIO()
df.to_csv(buffer, index=False, header=False)
buffer.seek(0)


with pg.connect(**DB_CONFIG) as conn:
    # The COPY operation needs to be within a WITH block
    with conn.cursor().copy("COPY five_minute (date, adj_close, close, high, low, open, volume, ticker) FROM STDIN WITH CSV") as copy:
        copy.write(buffer.getvalue())
    
    conn.commit()

['III.L',
 'ADM.L',
 'AAF.L',
 'ALW.L',
 'AAL.L',
 'ANTO.L',
 'AHT.L',
 'ABF.L',
 'AZN.L',
 'AUTO.L',
 'AV.L',
 'BA.L',
 'BARC.L',
 'BTRW.L',
 'BEZ.L',
 'BKG.L',
 'BP.L',
 'BATS.L',
 'BLND.L',
 'BT-A.L',
 'BNZL.L',
 'CNA.L',
 'CCH.L',
 'CPG.L',
 'CTEC.L',
 'CRDA.L',
 'DCC.L',
 'DGE.L',
 'DPLM.L',
 'EDV.L',
 'ENT.L',
 'EZJ.L',
 'EXPN.L',
 'FCIT.L',
 'FRES.L',
 'GAW.L',
 'GLEN.L',
 'GSK.L',
 'HLN.L',
 'HLMA.L',
 'HL.L',
 'HIK.L',
 'HSX.L',
 'HWDN.L',
 'HSBA.L',
 'IHG.L',
 'IMI.L',
 'IMB.L',
 'INF.L',
 'ICG.L',
 'IAG.L',
 'ITRK.L',
 'JD.L',
 'KGF.L',
 'LAND.L',
 'LGEN.L',
 'LLOY.L',
 'LMP.L',
 'LSEG.L',
 'MNG.L',
 'MKS.L',
 'MRO.L',
 'MNDI.L',
 'NG.L',
 'NWG.L',
 'NXT.L',
 'PSON.L',
 'PSH.L',
 'PSN.L',
 'PHNX.L',
 'PRU.L',
 'RKT.L',
 'REL.L',
 'RTO.L',
 'RMV.L',
 'RIO.L',
 'RR.L',
 'SGE.L',
 'SBRY.L',
 'SDR.L',
 'SMT.L',
 'SGRO.L',
 'SVT.L',
 'SHEL.L',
 'SMDS.L',
 'SMIN.L',
 'SN.L',
 'SPX.L',
 'SSE.L',
 'STAN.L',
 'STJ.L',
 'TW.L',
 'TSCO.L',
 'ULVR.L',
 'UU.L',
 'UTG.L',
 'VOD.L',
 'WEIR